In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

# -----------------------------
# 1. Dataset
# -----------------------------
eng_sentences = ["How are you?", "What is your name?"]
tam_sentences = ["<start> நீங்கள் எப்படி இருக்கிறீர்கள்? <end>",
                 "<start> உங்கள் பெயர் என்ன? <end>"]

# -----------------------------
# 2. Tokenize
# -----------------------------
eng_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
eng_tokenizer.fit_on_texts(eng_sentences)
eng_seq = eng_tokenizer.texts_to_sequences(eng_sentences)
eng_seq = tf.keras.preprocessing.sequence.pad_sequences(eng_seq, padding='post')

tam_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
tam_tokenizer.fit_on_texts(tam_sentences)
tam_seq = tam_tokenizer.texts_to_sequences(tam_sentences)
tam_seq = tf.keras.preprocessing.sequence.pad_sequences(tam_seq, padding='post')

# -----------------------------
# 3. Prepare Decoder Input & Target
# -----------------------------
decoder_input_seq = tam_seq[:, :-1]
decoder_target_seq = np.expand_dims(tam_seq[:, 1:], -1)

eng_vocab = len(eng_tokenizer.word_index) + 1
tam_vocab = len(tam_tokenizer.word_index) + 1

max_eng_len = eng_seq.shape[1]
max_tam_len = decoder_input_seq.shape[1]

latent_dim = 64

# -----------------------------
# 4. Encoder
# -----------------------------
encoder_inputs = Input(shape=(max_eng_len,))
enc_emb = Embedding(eng_vocab, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# -----------------------------
# 5. Decoder
# -----------------------------
decoder_inputs = Input(shape=(max_tam_len,))
dec_emb = Embedding(tam_vocab, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(tam_vocab, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# -----------------------------
# 6. Model
# -----------------------------
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

# -----------------------------
# 7. Train
# -----------------------------
model.fit([eng_seq, decoder_input_seq], decoder_target_seq,
          batch_size=2,
          epochs=300,
          verbose=0)

# -----------------------------
# 8. Inference Models
# -----------------------------
# Encoder
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder
dec_state_input_h = Input(shape=(latent_dim,))
dec_state_input_c = Input(shape=(latent_dim,))
dec_states_inputs = [dec_state_input_h, dec_state_input_c]

dec_emb2 = Embedding(tam_vocab, latent_dim, mask_zero=True)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=dec_states_inputs)
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model([decoder_inputs] + dec_states_inputs,
                      [decoder_outputs2, state_h2, state_c2])

# -----------------------------
# 9. Translate function
# -----------------------------
def translate(sentence):
    seq = eng_tokenizer.texts_to_sequences([sentence])
    seq = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=max_eng_len, padding='post')
    states = encoder_model.predict(seq)

    target_seq = np.zeros((1,1))
    target_seq[0,0] = tam_tokenizer.word_index['<start>']
    output_sentence = ""
    for _ in range(max_tam_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states)
        sampled_token_index = np.argmax(output_tokens[0,-1,:])
        word = [k for k,v in tam_tokenizer.word_index.items() if v==sampled_token_index][0]
        if word == '<end>':
            break
        output_sentence += word + " "
        target_seq[0,0] = sampled_token_index
        states = [h, c]
    return output_sentence.strip()

# -----------------------------
# 10. Test Translation
# -----------------------------
for s in eng_sentences:
    print("English:", s)
    print("Tamil (predicted):", translate(s))
    print()


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_9       │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_8         │ (None, 4, 64)     │        512 │ input_layer_8[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_4         │ (None, 4)         │          0 │ input_layer_8[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_9         │ (None, 4, 64)     │        576 │ input_layer_9[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 64),      │     33,024 │ embedding_8[0][0… │
│                     │ (None, 64),       │            │ not_equal_4[0][0] │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 4, 64),   │     33,024 │ embedding_9[0][0… │
│                     │ (None, 64),       │            │ lstm_4[0][1],     │
│                     │ (None, 64)]       │            │ lstm_4[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 4, 9)      │        585 │ lstm_5[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 67,721 (264.54 KB)

 Trainable params: 67,721 (264.54 KB)

 Non-trainable params: 0 (0.00 B)

English: How are you?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Tamil (predicted): நீங்கள் எப்படி இருக்கிறீர்கள்?

English: What is your name?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Tamil (predicted): உங்கள் பெயர் என்ன?

